In [1]:
!pip install --upgrade pip
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 57.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 80.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 82.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 82.7 MB/s eta 0:00:00
  Consider adding this directory to PATH

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

## data = [
    "x1;x1",
]

expressions = [line.split(';')[0] for line in data]
var_orders = [line.split(';')[1] for line in data]

# Tokenizing expressions
expr_tokenizer = Tokenizer(char_level=True)
expr_tokenizer.fit_on_texts(expressions)
expr_sequences = expr_tokenizer.texts_to_sequences(expressions)
expr_sequences_padded = pad_sequences(expr_sequences, padding='post')

# Tokenizing variable orders
var_tokenizer = Tokenizer(char_level=True, filters='', lower=False)
var_tokenizer.fit_on_texts(var_orders)
var_sequences = var_tokenizer.texts_to_sequences(var_orders)
var_sequences_padded = pad_sequences(var_sequences, padding='post')

# Convert to numpy arrays for training
X = np.array(expr_sequences_padded)
Y = np.array(var_sequences_padded)

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, TimeDistributed, Activation
from tensorflow.keras.activations import softmax

from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K

def pointer_network(input_dim, output_dim, max_input_len, max_output_len, embedding_dim, units):
    # Encoder
    encoder_input = Input(shape=(max_input_len,))
    encoder_embedding = Embedding(input_dim + 1, embedding_dim)(encoder_input)
    encoder_lstm, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_input = Input(shape=(max_output_len,))
    decoder_embedding = Embedding(output_dim + 1, embedding_dim)(decoder_input)
    decoder_lstm = LSTM(units, return_sequences=True)(decoder_embedding, initial_state=encoder_states)
    attention = Dense(max_input_len, activation='softmax')(decoder_lstm)
    pointer_layer = TimeDistributed(Dense(output_dim + 1, activation='linear'))(attention)
    output = Activation(softmax)(pointer_layer)

    model = Model([encoder_input, decoder_input], output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

ModuleNotFoundError: No module named 'tensorflow'

In [86]:
# Model parameters
input_dim = len(expr_tokenizer.word_index)
output_dim = len(var_tokenizer.word_index)
max_input_len = X.shape[1]
max_output_len = Y.shape[1]
embedding_dim = 32
units = 64

# Build model
model = pointer_network(input_dim, output_dim, max_input_len, max_output_len, embedding_dim, units)
model.summary()

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18      │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_19      │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_18        │ (None, 2, 32)     │         96 │ input_layer_18[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_19        │ (None, 2, 32)     │         96 │ input_layer_19[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_18 (LSTM)      │ [(None, 64),      │     24,832 │ embedding_18[0][… │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_19 (LSTM)      │ (None, 2, 64)     │     24,832 │ embedding_19[0][… │
│                     │                   │            │ lstm_18[0][1],    │
│                     │                   │            │ lstm_18[0][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 2, 2)      │        130 │ lstm_19[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_7  │ (None, 2, 3)      │          9 │ dense_18[0][0]    │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 2, 3)      │          0 │ time_distributed… │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 49,995 (195.29 KB)

 Trainable params: 49,995 (195.29 KB)

 Non-trainable params: 0 (0.00 B)

In [87]:
# Prepare decoder input data which is offset by one timestep
decoder_input_data = np.zeros_like(Y)
decoder_input_data[:, 1:] = Y[:, :-1]
decoder_input_data[:, 0] = 1  # Start token, assuming '1' is the index of start token

In [88]:
model.fit([X, decoder_input_data], tf.keras.utils.to_categorical(Y, num_classes=output_dim+1), epochs=500, batch_size=1)

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1.4071
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.0000e+00 - loss: 1.4031
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.0000e+00 - loss: 1.3991
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.0000e+00 - loss: 1.3950
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.0000e+00 - loss: 1.3909
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.0000e+00 - loss: 1.3867
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.0000e+00 - loss: 1.3825
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.0000e+00 - loss: 1.3781
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.0000e+00 - loss: 1.3736
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.0000e+00 - loss: 1.3689
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.0000e+00 - loss: 1.3640
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━

In [89]:
loss, accuracy = model.evaluate([X, decoder_input_data], tf.keras.utils.to_categorical(Y, num_classes=output_dim+1))
print(f"Loss: {loss}, Accuracy: {accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step - accuracy: 0.5000 - loss: 0.8355
Loss: 0.8355403542518616, Accuracy: 0.5


In [90]:
def predict_variable_order(expression, expr_tokenizer, var_tokenizer, model, max_input_len, max_output_len):
    # Tokenize and pad the input expression
    seq = expr_tokenizer.texts_to_sequences([expression])
    padded_seq = pad_sequences(seq, maxlen=max_input_len, padding='post')
    
    # Prepare decoder input (start with the start token)
    decoder_input = np.zeros((1, max_output_len))
    decoder_input[0, 0] = 1  # Assuming '1' is the index of the start token
    
    # Predict the output sequence
    predictions = model.predict([padded_seq, decoder_input])[0]
    
    # Convert predictions to variable order indices
    predicted_indices = np.argmax(predictions, axis=-1)
    
    # Convert indices to variable names
    index_to_word = {index: word for word, index in var_tokenizer.word_index.items()}
    predicted_vars = [index_to_word.get(idx, '') for idx in predicted_indices if idx > 0]  # Exclude padding
    
    # Join the predicted variable names into a string
    predicted_order = ','.join(predicted_vars)
    
    return predicted_order

In [91]:
expression = "(x3&x1|~x4)&x2;x1,x2,x3,x4"
predicted_order = predict_variable_order(expression, expr_tokenizer, var_tokenizer, model, max_input_len, max_output_len)
print(f"Predicted variable order for '{expression}': {predicted_order}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
Predicted variable order for '(x3&x1|~x4)&x2;x1,x2,x3,x4': x,x
